<h2>Import Libraries

In [4]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
import warnings
warnings.filterwarnings('ignore')

In [8]:
bucket_name = 'ml-dataset-01'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-2


In [9]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-2':
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})
        print('Bucket created succesfully')
except Exception as e:
    print("s3 exception", e)

Bucket created succesfully


In [11]:
#output path
prefix = 'xgboost-model'
output_path = f's3://{bucket_name}//{prefix}'
print(output_path)

s3://ml-dataset-01//xgboost-model


<h2> Download Dataset and storing in s3

In [14]:
import pandas as pd
import urllib
import numpy as np
try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print("Downloaded successfully")
except Exception as e:
    print("Data load error", e)
    
try:
    df = pd.read_csv('./bank_clean.csv', index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Downloaded successfully
Success: Data loaded into dataframe.


In [15]:
#train, test split
train_data, test_data = np.split(df.sample(frac=1, random_state=101), [int(0.75 * len(df))])
print(train_data.shape)
print(test_data.shape)

(30891, 61)
(10297, 61)


In [16]:
train_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
3669,32,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0
20131,57,2,999,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
2492,33,3,999,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0
14088,29,2,999,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
34986,27,5,999,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0


In [38]:
# saving train, test data in buckets
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_yes', 'y_no'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = s3_input(s3_data = f's3://{bucket_name}/{prefix}/train', content_type='csv')

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [39]:
pd.concat([test_data['y_yes'], test_data.drop(['y_yes', 'y_no'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = s3_input(s3_data = f's3://{bucket_name}/{prefix}/test', content_type='csv')

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


<h2> Building XG-boost inbuilt model

In [20]:
container = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [21]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [27]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(), 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge',
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [40]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2022-04-27 11:24:23 Starting - Starting the training job...
2022-04-27 11:24:25 Starting - Launching requested ML instancesProfilerReport-1651058663: InProgress
......
2022-04-27 11:25:49 Starting - Preparing the instances for training.........
2022-04-27 11:27:12 Downloading - Downloading input data
2022-04-27 11:27:12 Training - Downloading the training image.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[11:27:27] 30891x59 matrix with 1822569 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimi

In [41]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

------!

<h2>Prediction of the Test Data

In [43]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(10297,)


In [44]:
predictions_array

array([0.10349805, 0.08989331, 0.05087551, ..., 0.04245112, 0.01708536,
       0.06572308])

In [45]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.6%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (8970)    34% (135)
Purchase        9% (934)     66% (258) 



<h2>Deleting The Endpoints

In [46]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[{'ResponseMetadata': {'RequestId': '8S3N04VDBRF2QNPM',
   'HostId': 'XS2APJKhyorYOBTA97voKAjuvLDPx5lJ+Djb0LjNnKoo0HQ5I8Ho6/uhdDic4nnwNo8xgaXB7PQ=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'XS2APJKhyorYOBTA97voKAjuvLDPx5lJ+Djb0LjNnKoo0HQ5I8Ho6/uhdDic4nnwNo8xgaXB7PQ=',
    'x-amz-request-id': '8S3N04VDBRF2QNPM',
    'date': 'Wed, 27 Apr 2022 11:55:03 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': '/xgboost-model/sagemaker-xgboost-2022-04-27-11-24-23-291/output/model.tar.gz'},
   {'Key': '/xgboost-model/sagemaker-xgboost-2022-04-27-11-24-23-291/profiler-output/framework/training_job_end.ts'},
   {'Key': '/xgboost-model/sagemaker-xgboost-2022-04-27-11-24-23-291/profiler-output/system/incremental/2022042711/1651058820.algo-1.json'},
   {'Key': 'xgboost-model/train/train.csv'},
   {'Key': '/xgboost-model/sagemaker-xgboost-2022-04-27-11-24-2